# Spiking Neural Network Simulation with Seizure Transitions

This notebook demonstrates a spiking neural network model with excitatory (E) and two types of inhibitory neurons (PV, SST) that simulates transitions between normal, pre-seizure, and seizure states.

In [1]:
#Import necessary packages and set up environment
using DrWatson
findproject(@__DIR__) |> quickactivate

using SpikingNeuralNetworks
const SNN = SpikingNeuralNetworks
using UnPack
using Logging
using Plots
using Statistics

# Set global logger to display messages in console
global_logger(ConsoleLogger())

# Load units for physical quantities
SNN.@load_units

# Enable plotting in notebook
gr()
Plots.default(show=true)




## 1. Network Configuration

Define the network parameters including neuron populations, synaptic properties,
and connection probabilities.

In [ ]:
import SpikingNeuralNetworks: IF, PoissonLayer, Stimulus, SpikingSynapse, compose, monitor!, sim!, firing_rate, @update, SingleExpSynapse, IFParameter, Population, PostSpike, AdExParameter

PV_SST_microcircuit_network = (
    # Number of neurons in each population
    Npop = (E=4000, inh_PV=500, inh_SST=500),

    # Parameters for excitatory neurons
    exc = IFParameter(
        τm = 200pF / 10nS,
        El = -70mV,
        Vt = -50.0mV,
        Vr = -70.0mV,
        R  = 1/10nS,
    ),

    # PV interneurons (fast)
    inh_PV = IFParameter(
        τm = 100pF / 10nS,
        El = -70mV,
        Vt = -53.0mV,
        Vr = -70.0mV,
        R  = 1/10nS,
    ),

    # SST interneurons (slow)
    inh_SST = IFParameter(
        τm = 200pF / 10nS,
        El = -70mV,
        Vt = -53.0mV,
        Vr = -70.0mV,
        R  = 1/10nS,
    ),

    # Spike parameters
    spike_exc = PostSpike(τabs = 2ms),
    spike_pv  = PostSpike(τabs = 1ms),
    spike_sst = PostSpike(τabs = 2ms),

    # Synaptic dynamics
    synapse_exc = SingleExpSynapse(
        τe = 5ms,
        τi = 5ms,
        E_e = 0mV,
        E_i = -80mV,
    ),

    synapse_inh_fast = SingleExpSynapse(   # PV → E (fast, somatic)
        τi = 5ms,
        τe = 5ms,
        E_i = -80mV,
        E_e = 0mV,
    ),

    synapse_inh_slow = SingleExpSynapse(   # SST → E (slow, dendritic)
        τi = 25ms,
        τe = 5ms,
        E_i = -80mV,
        E_e = 0mV,
    ),

    # Connectivity
    connections = (
        E_to_E   = (p = 0.20, μ = 2nS,  rule=:Fixed),
        E_to_PV  = (p = 0.50, μ = 2nS,  rule=:Fixed),
        E_to_SST = (p = 0.50, μ = 2nS,  rule=:Fixed),

        PV_to_E   = (p = 0.80, μ = 10nS, rule=:Fixed),
        SST_to_E  = (p = 0.50, μ = 3nS,  rule=:Fixed),
        SST_to_PV = (p = 0.50, μ = 2nS,  rule=:Fixed),

        SST_to_SST = (p = 0.20, μ = 1nS, rule=:Fixed),
    ),

    # External Poisson input
    afferents = (
        layer = PoissonLayer(rate=10Hz, N=100),
        conn = (p = 0.1, μ = 4.0nS),
    ),
    )
end

UndefKeywordError: UndefKeywordError: keyword argument `A` not assigned

## 2.Network Construction

Define a function to create the network based on the configuration parameters.

In [ ]:
# Function to create the network
function build_network(config)
    @unpack afferents, connections, Npop, spike_exc, spike_pv, spike_sst = config
    @unpack exc, inh_PV, inh_SST, synapse_exc, synapse_inh_fast, synapse_inh_slow = config
    @unpack layer, conn = afferents

    # Create neuron populations
    E   = Population(exc;     synapse=synapse_exc,      spike=spike_exc, N=Npop.E,       name="E")
    PV  = Population(inh_PV;  synapse=synapse_inh_fast, spike=spike_pv,  N=Npop.inh_PV,  name="PV")
    SST = Population(inh_SST; synapse=synapse_inh_slow, spike=spike_sst, N=Npop.inh_SST, name="SST")

    # External Poisson background input to each population
    stimuli = (
        afferentE   = Stimulus(layer, E,   :glu, conn=conn, name="noiseE"),
        afferentPV  = Stimulus(layer, PV,  :glu, conn=conn, name="noisePV"),
        afferentSST = Stimulus(layer, SST, :glu, conn=conn, name="noiseSST"),
    )
    

    # Recurrent synaptic connections
    synapses = (
        E_to_E    = SpikingSynapse(E,  E,   :glu,  conn=connections.E_to_E,    name="E_to_E"),
        E_to_PV   = SpikingSynapse(E,  PV,  :glu,  conn=connections.E_to_PV,   name="E_to_PV"),
        E_to_SST  = SpikingSynapse(E,  SST, :glu,  conn=connections.E_to_SST,  name="E_to_SST"),

        PV_to_E   = SpikingSynapse(PV, E,   :gaba, conn=connections.PV_to_E,   name="PV_to_E"),
        SST_to_E  = SpikingSynapse(SST,E,   :gaba, conn=connections.SST_to_E,  name="SST_to_E"),
        SST_to_PV = SpikingSynapse(SST,PV,  :gaba, conn=connections.SST_to_PV, name="SST_to_PV"),
        SST_to_SST= SpikingSynapse(SST,SST, :gaba, conn=connections.SST_to_SST,name="SST_to_SST"),
    )

    # Compose into network model
    model = compose(; populations..., stimuli..., synapses..., name="PV_SST_Microcircuit")

    # Set up monitoring
    monitor!(model.pop, [:fire, :v])
    monitor!(model.stim, [:fire])

    return model, populations
end

build_network (generic function with 1 method)

## 3.Parameter Modulation for Seizure Transitions

Define parameters that can induce seizure-like activity.

In [ ]:
# Cache baseline synaptic weights (μ) once to avoid cumulative scaling errors
function cache_baseline!(model)
    base = Dict{Symbol,Any}()
    for k in (:E_to_E, :PV_to_E, :SST_to_E, :SST_to_PV)
        base[k] = deepcopy(model.syn[k].param.μ)
    end
    model.meta = get(model, :meta, Dict{Symbol,Any}())
    model.meta[:baseline_μ] = base
    return model
end

# Set a "state" by absolute scaling (relative to the cached baseline)
function set_state!(model; drive=10Hz, wEE=1.0, wPV_E=1.0, wSST_E=1.0, wSST_PV=1.0)
    # External Poisson drive to all three populations
    model.stim[:afferentE].param.rate = drive
    model.stim[:afferentPV].param.rate = drive
    model.stim[:afferentSST].param.rate = drive

    # Absolute (not cumulative) weight scaling
    base = model.meta[:baseline_μ]
    SNN.@update model.syn[:E_to_E].param  begin μ .= base[:E_to_E]  .* wEE      end
    SNN.@update model.syn[:PV_to_E].param begin μ .= base[:PV_to_E] .* wPV_E    end
    SNN.@update model.syn[:SST_to_E].param begin μ .= base[:SST_to_E].* wSST_E  end
    SNN.@update model.syn[:SST_to_PV].param begin μ .= base[:SST_to_PV].* wSST_PV end
    return nothing
end



configure_network_state! (generic function with 1 method)


## 4.Phase Protocol (Normal → Pre → Seizure )

**Purpose:** Declare the sequence of phases and a runner that applies each state and simulates for its duration.
 - Each phase adjusts `drive` and synaptic scales `(EE, PV_E, SST_E, SST_PV)`.


In [ ]:
#  Simulation Protocol
phases = [
    (name="normal", dur=1.5s, drive=10Hz, w=(EE=1.00, PV_E=1.00, SST_E=1.00, SST_PV=1.00)),
    (name="pre", dur=1.5s, drive=15Hz, w=(EE=1.20, PV_E=0.50, SST_E=1.20, SST_PV=1.20)),
    (name="seizure", dur=1.5s, drive=20Hz, w=(EE=1.50, PV_E=0.10, SST_E=1.30, SST_PV=1.30)),
]

## Simulation with Dynamic Parameter Changes
# Build network
model, populations = build_network(PV_SST_microcircuit_network)

# Monitor spikes + membrane potentials BEFORE simulating
monitor!(model.pop, [:fire, :v]) # Request :v for all populations
monitor!(model.stim, [:fire])

#  Cache baseline synaptic weights
cache_baseline!(model)

# Run the phase protocol
for ph in phases
    @info "Simulating phase: $(ph.name) for $(ph.dur)"
    set_state!(model; drive=ph.drive,
        wEE=ph.w.EE, wPV_E=ph.w.PV_E, wSST_E=ph.w.SST_E, wSST_PV=ph.w.SST_PV)
    SNN.sim!(model, duration=ph.dur)
end

run_phase_protocol (generic function with 1 method)

## 5.Analysis  (Rates & Balance)

 **Purpose:** Compute population-averaged firing rates and derive interpretable indicators:
 - **PV/SST ratio**: inhibitory subtype balance
 - **E/(PV+SST)**: net excitation–inhibition balance
 Also compute per-phase summary statistics (mean ± std).


 ## 6. Visualization (Rates, Indicators, Raster, Vm)

 **Purpose:** Create figures for:
 1) population firing rates with phase separators,
 2) PV/SST and E/I indicators,
 3) sampled raster plot,
 4) representative membrane potentials.